# MovieLens Example

In this notebook, we train an AverageModel on the MovieLens dataset with a BPRLoss.

The pipeline is made of 4 steps

- step 1: given the MovieLens ratings.csv file, create tfrecords for the training, evaluation and test sets.
- step 2: train an AverageModel (optionally, use tf-yarn to distribute training and evaluation on a cluster) and export the embeddings as well as the graph.
- step 3: write predictions, i.e. for all the test timelines, compute the user representation.
- step 4: evaluate predictions, i.e. look how similar the recommendations provided by the model are to the actual movies seen by the user in the "future"

## Install

In [ ]:
!pip install deepr[cpu] faiss_cpu

## Download the dataset

First download the movielens dataset

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip

In [ ]:
import os
import logging

import tensorflow as tf

import deepr as dpr
from deepr.examples import movielens

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
dataset_path = os.getcwd() + "/ml-20m"
path_ratings = f"{dataset_path}/ratings.csv"
path_root = "wan"
path_model = path_root + "/model"
path_data = path_root + "/data"
path_variables = path_root + "/variables"
path_predictions = path_root + "/predictions.parquet.snappy"
path_saved_model = path_root + "/saved_model"
path_mapping = path_data + "/mapping.txt"
path_train = path_data + "/train.tfrecord.gz"
path_eval = path_data + "/eval.tfrecord.gz"
path_test = path_data + "/test.tfrecord.gz"
dpr.io.Path(path_root).mkdir(exist_ok=True)
dpr.io.Path(path_model).mkdir(exist_ok=True)
dpr.io.Path(path_data).mkdir(exist_ok=True)
max_steps = 100_000

## Step 1 - Build TF Records

The job takes as input the csv ratings and create 3 tfrecords files (train, validation, test). 

Each file contains timeline of user ratings split into input and target.

In [4]:
build = movielens.jobs.BuildRecords(
    path_ratings=path_ratings,
    path_mapping=path_mapping,
    path_train=path_train,
    path_eval=path_eval,
    path_test=path_test,
    min_rating=4,
    min_length=5,
    num_negatives=8,
    target_ratio=0.2,
    size_test=10_000,
    size_eval=10_000,
    shuffle_timelines=True,
    seed=2020,
)

Now that the job is defined, let's run it. 

We run it before defining the other jobs because we need to know the vocabulary size (number of movies in the dataset) to build the embedding matrix.

In [ ]:
build.run()

## Step 2 - Training

This defines a training job to build a model that transform a timeline of movies embedding to an user embedding.
It takes as input the tf records and upon completion writes 3 artifacts :
* dataframe of the biases and the embeddings
* saved model : protobuf containing the model definition and weights

In this specific instance we train an average model with a BPR loss and compute a triple precision on the validation set.

In [7]:
train = dpr.jobs.Trainer(
    path_model=path_model,
    pred_fn=movielens.layers.AverageModel(vocab_size=dpr.vocab.size(path_mapping), dim=100),
    loss_fn=movielens.layers.BPRLoss(vocab_size=dpr.vocab.size(path_mapping), dim=100),
    optimizer_fn=dpr.optimizers.TensorflowOptimizer("LazyAdam", 0.001),
    train_input_fn=dpr.readers.TFRecordReader(path_train),
    eval_input_fn=dpr.readers.TFRecordReader(path_eval, shuffle=False),
    prepro_fn=movielens.prepros.RecordPrepro(
        min_input_size=3,
        min_target_size=3,
        max_input_size=50,
        max_target_size=50,
        buffer_size=1024,
        batch_size=128,
        repeat_size=None,
        prefetch_size=1,
        num_parallel_calls=8,
    ),
    train_spec=dpr.jobs.TrainSpec(max_steps=max_steps),
    eval_spec=dpr.jobs.EvalSpec(steps=None, start_delay_secs=30, throttle_secs=30),
    final_spec=dpr.jobs.FinalSpec(steps=None),
    exporters=[
        # The training will keep the model with the best triplet precision
        dpr.exporters.BestCheckpoint(metric="triplet_precision", mode="increase"),
        # Export biases and embeddings as a dataframe
        dpr.exporters.SaveVariables(path_variables=path_variables, variable_names=["biases", "embeddings"]),
        # Export a saved model using specified fields as input
        dpr.exporters.SavedModel(
            path_saved_model=path_saved_model,
            fields=[
                dpr.Field(name="inputPositives", shape=(None,), dtype=tf.int64),
                dpr.Field(name="inputMask", shape=(None,), dtype=tf.bool),
            ],
        ),
    ],
    train_hooks=[
        # Log metrics, hyperparams, initial values to the console, and optionally mlflow and graphite
        dpr.hooks.LoggingTensorHookFactory(
            name="training",
            functions={
                "memory_gb": dpr.hooks.ResidentMemory(unit="gb"),
                "max_memory_gb": dpr.hooks.MaxResidentMemory(unit="gb"),
            },
            every_n_iter=300,
            use_graphite=False,
            use_mlflow=False,
        ),
        dpr.hooks.SummarySaverHookFactory(save_steps=300),
        dpr.hooks.NumParamsHook(use_mlflow=False),
        dpr.hooks.LogVariablesInitHook(use_mlflow=False),
        dpr.hooks.StepsPerSecHook(
            name="training",
            batch_size=128,
            every_n_steps=300,
            skip_after_step=max_steps,
            use_mlflow=False,
            use_graphite=False,
        ),
        # Stop the training if triplet precision does not improve
        dpr.hooks.EarlyStoppingHookFactory(
            metric="triplet_precision",
            mode="increase",
            max_steps_without_improvement=1000,
            min_steps=5_000,
            run_every_steps=300,
            final_step=max_steps,
        ),
    ],
    eval_hooks=[dpr.hooks.LoggingTensorHookFactory(name="validation", at_end=True)],
    final_hooks=[dpr.hooks.LoggingTensorHookFactory(name="final_validation", at_end=True)],
    train_metrics=[dpr.metrics.StepCounter(name="num_steps"), dpr.metrics.DecayMean(tensors=["loss"], decay=0.98)],
    eval_metrics=[dpr.metrics.Mean(tensors=["loss", "triplet_precision"])],
    final_metrics=[dpr.metrics.Mean(tensors=["loss", "triplet_precision"])],
    run_config=dpr.jobs.RunConfig(
        save_checkpoints_steps=300, save_summary_steps=300, keep_checkpoint_max=None, log_step_count_steps=300
    ),
    config_proto=dpr.jobs.ConfigProto(
        inter_op_parallelism_threads=8, intra_op_parallelism_threads=8, gpu_device_count=0, cpu_device_count=48,
    ),
)

## Step 3 - Predict job

The predict job reloads the test dataset and the saved model, performs the inference and writes the user embeddings in a dataframe to be reused later by the validation job.

In [8]:
predict = movielens.jobs.Predict(
    path_saved_model=path_saved_model,
    path_predictions=path_predictions,
    input_fn=dpr.readers.TFRecordReader(path_test, shuffle=False),
    prepro_fn=movielens.prepros.RecordPrepro(),
)

## Step 4 - Evaluation job

This job takes user embedding, and uses faiss to retrieve the k nearest neighboors in the product embedding space, 
and compute metrics using the target timelines.

In [9]:
evaluate = [
    movielens.jobs.Evaluate(
        path_predictions=path_predictions,
        path_embeddings=path_variables + "/embeddings",
        path_biases=path_variables + "/biases",
        k=k,
    )
    for k in [10, 20, 50]
]

## Run the pipeline
All the jobs definition are lazy, and so is the pipeline. 
Calling run on it will actually perform all these steps.

In [18]:
pipeline = dpr.jobs.Pipeline([train, predict] + evaluate)

In [19]:
pipeline.run()

INFO:deepr.examples.movielens.jobs.evaluate:precision@10 = 0.09155
recall@10 = 0.11125661123870513
f1@10 = 0.08535778342371855
INFO:deepr.examples.movielens.jobs.evaluate:precision@20 = 0.08529500000000001
recall@20 = 0.20106905522924376
f1@20 = 0.103350595200071
INFO:deepr.examples.movielens.jobs.evaluate:precision@50 = 0.06869600000000001
recall@50 = 0.37075841696015943
f1@50 = 0.10448758409036087


## Visualize embeddings with a KNN

Let's check if the movie embeddings produce make sense
* load the movie embeddings
* load the movie title
* build a knn index on that
* do a query with a known movie to check that its closest neighboors make sense

In [15]:
import numpy as np
import pyarrow.parquet as pq
import faiss
import pyarrow.csv as pc
from IPython.display import display
import pandas as pd

In [16]:
embeddings = np.vstack(pq.read_table("wan/variables/embeddings").to_pandas().to_numpy())
mapp = {int(movie_id):indice for indice, movie_id in enumerate(open("wan/data/mapping.txt", "r").read().split("\n"))}
inversed_map = {indice:movie_id for movie_id, indice in mapp.items()}
index = faiss.IndexFlatIP(embeddings.shape[-1])
index.add(np.ascontiguousarray(embeddings))

def knn_query(index, query, ksearch):
    D, I = index.search(np.expand_dims(query,0), ksearch)
    distances = D[0]
    product_indices = I[0]
    product_ids = [inversed_map[i] for i in product_indices]
    return list(zip(product_ids, distances))

movies = pc.read_csv(f"{dataset_path}/movies.csv").to_pandas()
    
def display_results_df(results):
    data = [[movies[movies.movieId == movie_id]["genres"].to_numpy()[0], movies[movies.movieId == movie_id]["title"].to_numpy()[0], distance] for movie_id, distance in results]
    
    df = pd.DataFrame(data, columns = ['Genre', 'Title', 'Distance']) 
    display(df)
    
def display_movie(movie_id):
    data = [[(movies[movies.movieId == movie_id]["genres"].to_numpy()[0]), (movies[movies.movieId == movie_id]["title"].to_numpy()[0])]]
    
    df = pd.DataFrame(data, columns = ['Genre', 'Title']) 
    display(df)

In [17]:
p = movies[movies.title.str.lower().str.contains("star wars")]["movieId"].to_numpy()[0]
print("Query")
display_movie(p)
print("Knn results")
display_results_df(knn_query(index, embeddings[mapp[p]], 5))

Query


,Genre,Title
0,Action|Adventure|Sci-Fi,Star Wars: Episode IV - A New Hope (1977)


Knn results


,Genre,Title,Distance
0,Action|Adventure|Sci-Fi,Star Wars: Episode IV - A New Hope (1977),20.254013
1,Action|Adventure|Sci-Fi,Star Wars: Episode VI - Return of the Jedi (1983),18.168583
2,Action|Adventure|Sci-Fi,Star Wars: Episode V - The Empire Strikes Back...,17.663177
3,Action|Adventure,Raiders of the Lost Ark (Indiana Jones and the...,15.748468
4,Action|Adventure|Sci-Fi|Thriller,Star Trek: First Contact (1996),14.745905


As you can see, the genre of the nearest neighboors is similar and the movies are related to the movie query